# Invariance and Representation in Machine Learning

This notebook demonstrates how **input representation** (feature engineering) critically impacts machine learning performance. You will explore this concept using a simple, interpretable geometric dataset: 2D triangles.

## Learning Objectives
1.  **Feature Representation**: Why raw data (e.g., coordinates) isn't always the best input.
2.  **Invariance**: How removing "nuisance" variations (rotation, translation) simplifies learning.
3.  **Target Dependence**: A representation good for *area* might be bad for *position*.

## The Task
Predict three targets from triangle coordinates $(x_1, y_1), (x_2, y_2), (x_3, y_3)$:
1.  **$u$ (Position)**: Centroid sum ($x_c + y_c$). Depends on absolute position.
2.  **$v$ (Area)**: Triangle area. Depends on shape & scale.
3.  **$w$ (Shape)**: Ratio of inradius to circumradius ($r/R$). Depends only on intrinsic shape.

In [ ]:
# @title 1. Setup & Dependencies
# Run this cell to install necessary packages and import libraries.

!pip install -q numpy pandas scikit-learn matplotlib seaborn


In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Models
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from random import random

sns.set(style='whitegrid')
np.random.seed(42)
print("Setup complete.")

Setup complete.


In [ ]:
# @title 2. Data Generation
# This function generates random triangles and calculates u, v, w.
import random as rand_module


def area_triangle(x1, y1, x2, y2, x3, y3):
    return 0.5 * abs(x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2))

def centroid_sum(x1, y1, x2, y2, x3, y3):
    return (x1 + x2 + x3)/3.0 + (y1 + y2 + y3)/3.0

def get_angles(x1, y1, x2, y2, x3, y3):
    """Get angles (still needed for preprocessing mode='angle')."""
    a = np.hypot(x2-x3, y2-y3)
    b = np.hypot(x1-x3, y1-y3)
    c = np.hypot(x1-x2, y1-y2)
    if a==0 or b==0 or c==0: return float('nan')
    # Law of cosines
    def safe_acos(val):
        return math.acos(max(-1.0, min(1.0, val)))
    A = safe_acos((b**2 + c**2 - a**2)/(2*b*c))
    B = safe_acos((a**2 + c**2 - b**2)/(2*a*c))
    C = safe_acos((a**2 + b**2 - c**2)/(2*a*b))
    return A, B, C

def inradius_circumradius_ratio(x1, y1, x2, y2, x3, y3):
    """Compute ratio of inradius (r) to circumradius (R).
    r = Area / s (s = semi-perimeter)
    R = (a * b * c) / (4 * Area)
    r / R = 4 * Area^2 / (s * a * b * c)
    """
    a = np.hypot(x2-x3, y2-y3)
    b = np.hypot(x1-x3, y1-y3)
    c = np.hypot(x1-x2, y1-y2)
    if a==0 or b==0 or c==0: return float('nan')
    area = area_triangle(x1, y1, x2, y2, x3, y3)
    if area == 0: return float('nan')
    s = (a + b + c) / 2.0
    return (4.0 * area * area) / (s * a * b * c)

def generate_dataset(n=2000, noise=0.1):
    data = []
    for _ in range(n):
        # Generate random coordinates in [-10, 10]
        coords = np.random.uniform(-10, 10, 6)
        x1, y1, x2, y2, x3, y3 = coords

        v = area_triangle(x1, y1, x2, y2, x3, y3)
        if v < 1e-3: continue # Skip degenerate triangles

        u = centroid_sum(x1, y1, x2, y2, x3, y3)
        w = inradius_circumradius_ratio(x1, y1, x2, y2, x3, y3)
        if np.isnan(w): continue

        u += rand_module.gauss(0, noise)
        v += rand_module.gauss(0, noise)
        w += rand_module.gauss(0, noise)

        row = {'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'x3':x3, 'y3':y3, 'u':u, 'v':v, 'w':w}
        data.append(row)

    return pd.DataFrame(data)

df = generate_dataset(n=2000, noise=0.1)
print(f"Generated {len(df)} samples.")
df.head()

In [ ]:
# @title 3. Helper Functions: Training & Preprocessing

REGRESSORS = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(),
    'MLP (Neural Net)': MLPRegressor(hidden_layer_sizes=(64,32), max_iter=1000, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=50, random_state=42)
}

def preprocess_data(df, mode='none'):
    # Extract raw coords
    X_raw = df[['x1','y1','x2','y2','x3','y3']].values

    if mode == 'none':
        return pd.DataFrame(X_raw, columns=['x1','y1','x2','y2','x3','y3'])

    elif mode == 'congruent':
        # Translate p1 to origin, rotate p2 to x-axis
        new_X = []
        for row in X_raw:
            x1,y1,x2,y2,x3,y3 = row
            # Translate
            tx2, ty2 = x2-x1, y2-y1
            tx3, ty3 = x3-x1, y3-y1
            # Rotate
            theta = math.atan2(ty2, tx2)
            c, s = math.cos(-theta), math.sin(-theta)
            rx2 = tx2*c - ty2*s
            ry2 = tx2*s + ty2*c # Should be 0
            rx3 = tx3*c - ty3*s
            ry3 = tx3*s + ty3*c
            # Flip if y3 < 0 to handle reflection
            if ry3 < 0: ry3 = -ry3
            new_X.append([rx2, rx3, ry3])
        return pd.DataFrame(new_X, columns=['x2_new','x3_new','y3_new'])

    elif mode == 'similar':
        # Same as congruent but scale so p2 is at (1,0)
        new_X = []
        for row in X_raw:
            x1,y1,x2,y2,x3,y3 = row
            # Translate
            tx2, ty2 = x2-x1, y2-y1
            tx3, ty3 = x3-x1, y3-y1
            # Rotate
            dist_p1p2 = math.hypot(tx2, ty2)
            theta = math.atan2(ty2, tx2)
            scale = 1.0 / dist_p1p2 if dist_p1p2 != 0 else 0
            c, s = math.cos(-theta), math.sin(-theta)
            # Apply rotation & scale to p3
            rx3 = (tx3*c - ty3*s) * scale
            ry3 = (tx3*s + ty3*c) * scale
            if ry3 < 0: ry3 = -ry3
            new_X.append([rx3, ry3])
        return pd.DataFrame(new_X, columns=['x3_sim','y3_sim'])

    elif mode == 'length':
        # Edge lengths
        new_X = []
        for row in X_raw:
            x1,y1,x2,y2,x3,y3 = row
            a = math.hypot(x2-x3, y2-y3)
            b = math.hypot(x1-x3, y1-y3)
            c = math.hypot(x1-x2, y1-y2)
            new_X.append([a,b,c])
        return pd.DataFrame(new_X, columns=['len_a','len_b','len_c'])

    elif mode == 'angle':
        # Two angles
        new_X = []
        for row in X_raw:
            x1,y1,x2,y2,x3,y3 = row
            A, B, C = get_angles(x1,y1,x2,y2,x3,y3)
            new_X.append([A, B])
        return pd.DataFrame(new_X, columns=['angle_A','angle_B'])

def run_experiment(df, mode='none'):
    X = preprocess_data(df, mode)
    y = df[['u', 'v', 'w']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = []
    print(f"--- Results for Mode: {mode} ---")
    print(f"{'-'*60}")
    print(f"{'Target':<10} | {'Model':<15} | {'R2 Score':<10} | {'MSE':<10}")
    print(f"{'-'*60}")

    for target in ['u', 'v', 'w']:
        for name, model in REGRESSORS.items():
            pipe = make_pipeline(StandardScaler(), model)
            pipe.fit(X_train, y_train[target])
            pred = pipe.predict(X_test)
            r2 = r2_score(y_test[target], pred)
            mse = mean_squared_error(y_test[target], pred)
            results.append({'mode': mode, 'target': target, 'model': name, 'r2': r2, 'mse': mse})

            if r2 > 0.5: color = '\033[92m' # Green for good
            elif r2 > 0: color = '\033[93m' # Yellow for okay
            else: color = '\033[91m'       # Red for bad

            print(f"{target:<10} | {name:<15} | {color}{r2:.4f}\033[0m     | {mse:.4f}")
        print(f"{'-'*60}")
    return pd.DataFrame(results)

## Experiment 1: Raw Coordinates (`none`)
**Hypothesis:**
- $u$ (Position): Should be easy (Linear). $u$ is literally a linear combination of inputs.
- $v$ (Area): Should be harder. Area is non-linear (multiplicative).
- $w$ (Inradius/Circumradius): Should be hard. $r/R$ involves non-linear geometric relationships.

In [ ]:
res_none = run_experiment(df, mode='none')

## Experiment 2: Invariance (`congruent`)
We translate the triangle so $p_1$ is at $(0,0)$ and rotate it so $p_2$ is on the x-axis.

**Hypothesis:**
- **Position information is destroyed.** $u$ prediction should fail.
- **Area and Shape information is preserved.** $v$ and $w$ should improve or stay good because the model doesn't have to learn rotation invariance.

In [ ]:
res_congruent = run_experiment(df, mode='congruent')

## Experiment 3: Scale Invariance (`similar`)
We translate, rotate, AND scale the triangle so the base length is 1.

**Hypothesis:**
- **Scale information is destroyed.** Area $v$ depends on scale, so prediction should fail.
- **Shape information is preserved.** $r/R$ ($w$) only depends on shape, so it should still be predictable.

In [ ]:
res_similar = run_experiment(df, mode='similar')

## Experiment 4: Explicit Features (`length`)
We replace coordinates with the three edge lengths $(a, b, c)$.

**Hypothesis:**
- Similar to `congruent`, but explicitly provides the geometry the area formula needs (Heron's formula).
- $v$ (Area) prediction should be very strong.

In [ ]:
res_length = run_experiment(df, mode='length')

## Summary & Analysis


In [ ]:
all_results = pd.concat([res_none, res_congruent, res_similar, res_length])

# Filter for best model per mode/target (max R2)
best_results = all_results.loc[all_results.groupby(['mode', 'target'])['r2'].idxmax()]

plt.figure(figsize=(12, 6))
sns.barplot(data=best_results, x='target', y='r2', hue='mode')
plt.title("Best R2 Score per Target by Representation")
plt.ylim(-0.1, 1.1)
plt.axhline(0, color='black', linewidth=1)
plt.ylabel("R2 Score (Higher is better)")
plt.show()